In [2]:
import torch
from torch import nn
import numpy as np


## Transposed Conv.

output size = s(n - 1) + k -2p<br/>

s: stride<br/>
n: input height/width<br/>
k: kernel size<br/>
p: padding<br/>

Increase p -> Decrease output size

In [14]:
in_channels = 1
out_channels = 2
in_size = 5
kernel_size = 4
stride = 3
pad = 1
bias = True

X = torch.ones(in_size*in_size*in_channels).view(1,in_channels,in_size,in_size)

conv_t = nn.ConvTranspose2d(in_channels=in_channels,
                            out_channels=out_channels,
                            stride=stride,
                            kernel_size=(kernel_size,kernel_size),
                            padding=pad,
                            bias=bias,
                            )
conv_t.weight.data.fill_(1)
with torch.no_grad():
    conv_t.bias.fill_(1)

print(conv_t(X).size())
print(conv_t(X))

torch.Size([1, 2, 14, 14])
tensor([[[[2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [3., 3., 5., 3., 3., 5., 3., 3., 5., 3., 3., 5., 3., 3.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [3., 3., 5., 3., 3., 5., 3., 3., 5., 3., 3., 5., 3., 3.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [3., 3., 5., 3., 3., 5., 3., 3., 5., 3., 3., 5., 3., 3.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [3., 3., 5., 3., 3., 5., 3., 3., 5., 3., 3., 5., 3., 3.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.],
          [2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2., 3., 2., 2.]],

         [[2., 2., 